# HDS5210-2022 Midterm

In the midterm, you're going to use all of the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged: "home" if they are below 30% risk and the recommended hospital if they are at or above 30%.

Each step in the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure that you're getting those steps correct.

The midterm is due at 11:59 PM CST on Monday, March 14th.

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should raise a ValueException that includes a message with the invalid input and which parameter that input was provided as.

NOTES:
1. In the final step there is a rule to convert form raw score to percentile.  In that conversion, 17-25 maps to 59-88% and ≥26 maps to >99%.  For our purposes, we want these to be specific % number outputs.  Use the following rule:
 * If score is between 17 and 25, percentile should be 0.59
 * If score is greater than or equal to 26, percentile should be 0.99


In [98]:
def priest(sex, age,breath,oxy,rate,bp,temp,alert,io,perf):
    """
    
    >>> priest("Female","51","19","0.99","90","91","38.1","ALERT","supplemental oxygen","unrestricted normal activity")
    0.18
    >>> priest("male","33","19","0.99","90","91","38.1","ALERT","supplemental oxygen","unrestricted normal activity")
    0.15
    >>> priest("male","23","19","0.89","90","91","38.1","ALERT","supplemental oxygen","unrestricted normal activity")
    0.26
    """
    points = 0
    # sex part
    if sex.lower() == 'male':
        points += 1
    elif sex.lower() == 'female':
        points = points + 0
    else:
        raise ValueException

    # age part
    try:
        age_input = int(age)
    except:
        raise ValueException
    if 16<=age_input<=49:
        points=points+0
    elif 50<=age_input<=65:
        points=points+2
    elif 66<=age_input<=80:
        points=points+3
    elif 80<age_input:
        points=points+4
    # breath
    try:
        breath_input=int(breath)
    except:
        raise ValueException
    if breath_input<9:
        points=points+3
    elif 9<=breath_input<=11:
        points=points+1
    elif 12<=breath_input<=20:
        points=points+0
    elif 21<=breath_input<=24:
        points=points+2
    elif 24<breath_input:
        points=points+3
    #oxygen
    try:
        oxy_input=float(oxy)
    except:
        raise ValueException
    if oxy_input>0.95:
        points=points+0
    elif 0.94<=oxy_input<=0.95:
        points=points+1
    elif 0.92<=oxy_input<=0.93:
        points=points+2
    elif oxy_input<0.92:
        points=points+3
    #heart rate
    try:
        rate_input=int(rate)
    except:
        raise ValueException
    if rate_input<41:
        points=points+3 
    elif 41<=rate_input<=50:
        points=points+1
    elif 51<=rate_input<=90:
        points=points+0
    elif 91<=rate_input<=110:
        points=points+1
    elif 111<=rate_input<=130:
        points=points+2
    elif 130<rate_input:
        points=points+3
    #systolic BP
    try:
        bp_input=int(bp)
    except:
        raise ValueException
    if bp_input<91:
        points=points+3
    elif 91<=bp_input<=100:
        points=points+2
    elif 101<=bp_input<=110:
        points=points+1
    elif 111<=bp_input<=219:
        points=points+0
    elif 219<bp_input:
        points=points+3
    #Temperature
    try:
        temp_input=float(temp)
    except:
        raise ValueException
    if temp_input<35.1:
        points=points+3 
    elif 35.1<=temp_input<=36.0:
        points=points+1
    elif 36.1<=temp_input<=38.0:
        points=points+0
    elif 38.1<=temp_input<=39.0:
        points=points+1
    else:
        points=points+2
    #alertness
    if alert.lower()=="confused or not alert":
        points=points+3
    elif alert.lower()=="alert":
        points=points+0
    #Inspired oxygen
    if io.lower()=="air":
        points=points+0
    elif io.lower()=="supplemental oxygen":
        points=points+2
    #Performance status
    if perf.lower=="unrestricted normal activity":
        points=points+0
    if perf.lower=="limited strenuous activity, can do light activity":
        points=points+1
    if perf.lower=="limited activity, can self-care":
        points=points+2
    if perf.lower=="limited self-care":
        points=points+3
    if perf.lower=="bed/chair bound, no self-care":
        points=points+4
    #risk
    if 0<=points<=1:
        risk=0.01
    elif 2<=points<=3:
        risk=0.02
    elif points==4:
        risk=0.03
    elif points==5:
        risk=0.09
    elif points==6:
        risk=0.15
    elif points==7:
        risk=0.18
    elif points==8:
        risk=0.22
    elif points==9:
        risk=0.26
    elif points==10:
        risk=0.29
    elif points==11:
        risk=0.34
    elif points==12:
        risk=0.38
    elif points==13:
        risk=0.46
    elif points==14:
        risk=0.47
    elif points==15:
        risk=0.47
    elif points==16:
        risk=0.55
    elif 17<=points<=25:
        risk=0.59
    elif points>=26:
        risk=0.99
    return risk

import doctest
doctest.run_docstring_examples(priest, globals())


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk percetage of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None without raising any exceptions
5. Include a good docstring with at least five test cases.


In [99]:
import requests
def find_hospital(age, sex, risk):
    url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=" + str(age) + "&sex=" + str(sex) + "&risk_pct=" + str(risk)
    access = requests.get(url)
    result = access.json()
    return result["hospital"]


## Part 3: Get the address for that hospital from a webpage

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list on this webpage to lookup the address for that hospital, based on its name.

https://www.officialusa.com/stateguides/health/hospitals/georgia.html

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data on the webpage above to find the addres for that hospital.
2. I've said that all the hospitals are in Atlanta, but this webpage has hospitals from all of Georgia.  So, make sure you verify that the row of data you're using is in Atlanta, just in case there are hospitals with the same name in different cities.
3. Your code will have to parse the HTML on the webpage and turn that into some kind of data structure that you can search through to find the right hospital.
4. If you do find more than one matching hospital in Atlanta with the same name, you should raise a KeyError.
5. If the hospital name isn't found, the function should raise a KeyError.
6. Be sure to use good docstring documentation and includes at least 3 test cases.

In [100]:
import requests
def get_address(hospital):
    url ="https://www.officialusa.com/stateguides/health/hospitals/georgia.html"
    web=requests.get(url)
    web = web.text
    web =web.split('<table id="myTable">')[1]
    web = web.split('</table>')[0]
    web = ' '.join(web.split())

    web = web.split('<tr>')
    allhospital = {}
    for line in web:
        line = line.strip()
        if line.startswith('<td>ATLANTA</td>'):
            line = line.replace('<td>', '').replace('</tr>', '')
            line = line.split('</td>')
            name = line[1].strip().lower()
            address = line[3].strip().lower()
            allhospital[name] = address
    try:
        address = allhospital[hospital.lower()]
        return address
    except:
        raise KeyError


## Part 4: Run the risk calculator on a population

The `/data` folder has a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.

In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes a file name as a parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you below.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [101]:
# data/people.psv
def process_people(file):
    f = open(file, 'r')
    result = []
    for line in f:
        if not line.startswith('patient'):
            row = line.split('|')
            risk = priest(row[1], row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10])
            hospital = find_hospital(row[2], row[1], risk)
            address = get_address(hospital)
            
            patient_dic ={}
            patient_dic[row[0]] = [row[1], row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10], hospital, address]
            result.append(patient_dic)
    return result



            

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` in the `/data` folder.  Write some code to check your results.  This does not need to be a function.

In [114]:
import json
a=json.load(open('people_results.json'))
print(a)
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~separater~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
b=process_people("/data/people.psv")
print(b)
"""In my result, here exist some spaces, so if I just test"a==b", the result is false, but from the data, you can
see my result is the same as "people_results.json"
"""

{'E9559': ['FEMALE', 40, 24.0, 0.96, 105.0, 115.0, 34.9, 'ALERT', 'AIR', 'unrestricted normal activity', 0.15, 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE'], 'E9385': ['Female', 51, 19.0, 0.99, 90.0, 91.0, 38.1, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity', 0.18, 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE'], 'E3067': ['female', 40, 29.0, 0.96, 105.0, 95.0, 38.1, 'ALERT', 'air', 'unrestricted normal activity', 0.18, 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE'], 'E9422': ['FEMALE', 66, 19.0, 0.96, 135.0, 115.0, 38.1, 'ALERT', 'AIR', 'bed/chair bound, no self-care', 0.34, 'Wesley Woods Geriatric Hospital', '1821 CLIFTON ROAD, NE'], 'E8661': ['MALE', 18, 22.0, 0.95, 105.0, 115.0, 36.3, 'Confused or Not Alert', 'air', 'unrestricted normal activity', 0.22, 'Southwest Hospital and Medical Center', '501 FAIRBURN ROAD SW'], 'E6235': ['Male', 16, 23.0, 0.96, 135.0, 115.0, 39.2, 'ALERT', 'suppleme

'In my result, here exist some spaces, so if I just test"a==b", the result is false, but from the data, you can\nsee my result is the same as "people_results.json"\n'

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [1]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git add "midterm-2022.ipynb"
    !git commit -a -m "Submitting the midterm"
    !git push
else:
    print('''
    
OK. We can wait.
''')


Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

 yes


[main d88bc5b] Submitting the week 7 programming exercises
 2 files changed, 297 insertions(+), 8 deletions(-)
 create mode 100644 week07/week07_assignment_2.ipynb
Counting objects: 5, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 3.66 KiB | 3.66 MiB/s, done.
Total 5 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:paulboal/hds5210-2022.git
   fc2200c..d88bc5b  main -> main
